In [39]:
from utilities.feed import get_stock_info
import utilities.variables as variables 
import yfinance as yf
import pandas as pd
import importlib
import utilities.parser as parser
import utilities.esg_service as esg_service
importlib.reload(parser)
importlib.reload(esg_service)

<module 'utilities.esg_service' from '/Users/herbishtini/Documents/UNI/Master Thesis/sustainability_portfolio_optimisation/utilities/esg_service.py'>

In [40]:
# esg_service.generate_esg_data()
#esg_service.generate_esg_data_for_each_industry()

In [65]:
ticker = yf.Ticker('BULTEN')
ticker.info['marketCap']

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BULTEN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BULTEN&crumb=428nSXWA2ml


KeyError: 'marketCap'

In [62]:
msft.info['marketCap']

3379471319040

In [51]:
msft.capital_gains

Series([], dtype: object)

In [57]:
tickers = yf.Tickers('msft aapl goog')
tickers.tickers['msft'].info

KeyError: 'msft'

In [56]:
pd.DataFrame(hist)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-06-11 00:00:00-04:00,13.13,13.32,12.99,13.16,746200,0.0,0.0
2024-06-12 00:00:00-04:00,13.80,14.38,13.53,14.06,960400,0.0,0.0
2024-06-13 00:00:00-04:00,14.06,14.09,13.28,13.42,737800,0.0,0.0
2024-06-14 00:00:00-04:00,13.14,13.40,13.02,13.18,716800,0.0,0.0
2024-06-17 00:00:00-04:00,13.11,13.25,13.00,13.10,665500,0.0,0.0
2024-06-18 00:00:00-04:00,13.05,13.20,13.00,13.01,586100,0.0,0.0
2024-06-20 00:00:00-04:00,13.00,13.16,12.93,13.01,458300,0.0,0.0
2024-06-21 00:00:00-04:00,13.05,13.26,12.99,13.17,1464100,0.0,0.0
2024-06-24 00:00:00-04:00,13.20,13.33,13.09,13.17,507000,0.0,0.0


In [ ]:
msft.news